In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Bidirectional, SimpleRNN
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import regularizers
from nltk.stem import ISRIStemmer
import string, re, emoji, Stemmer
import pyarabic.araby as ar
from nltk.tokenize import RegexpTokenizer
from sklearn.metrics import accuracy_score, classification_report

In [2]:
file_path = "train.xlsx"
df_train = pd.read_excel(file_path)
df_train

,review_description,rating
0,شركه زباله و سواقين بتبرشم و مفيش حتي رقم للشك...,-1
1,خدمة الدفع عن طريق الكي نت توقفت عندي اصبح فقط...,1
2,تطبيق غبي و جاري حذفه ، عاملين اكواد خصم و لما...,-1
3,فعلا تطبيق ممتاز بس لو فى امكانية يتيح لمستخدم...,1
4,سيء جدا ، اسعار رسوم التوصيل لا تمت للواقع ب ص...,-1
...,...,...
32031,التطبيق اصبح سيء للغايه نقوم بطلب لا يتم وصول ...,-1
32032,y love you,1
32033,الباقه بتخلص وبشحن مرتين باقه اضافيه ١٠٠ جنيه,-1
32034,تطبيق فاشل وصلني الطلب ناقص ومش ينفع اعمل حاجة...,-1


In [3]:
arabic_stop_words = [
    "و", "في", "من", "على", "إلى", "لا", "أو", "هو", "هي", "يكون",
    "أنا", "أنت", "هو", "هي", "نحن", "أنتم", "هم",
    "عن", "مع", "كما", "مثل", "بين", "إذا", "حتى", "منذ",
    "و", "أو", "لكن", "إذا", "إن",
    "اليوم", "غداً", "الآن", "ثم", "بعد",
    "كان", "يكون", "أصبح", "صار", "ليس", "لم",
    "هذا", "هذه", "ذلك", "تلك", 
    "كل", "على", "فيه", "منه", "عنه", "له", "به", "إليه", "لها", "فيها",
    "بها", "منها", "عنها", "إليها", "الذي", "التي", "اللذين", "اللذان", "اللتان",
    "اللتين", "هؤلاء", "ذلك", "هذه", "هذا", "تلك", "تحت", "فوق", "معه", "لديه",
    "عليه", "عليها", "أي", "هل", "إذا", "ماذا", "هناك", "هنالك", "إلى",
    "يناير", "فبراير", "مارس", "إبريل", "مايو", "يونيو", "يوليو", "أغسطس", "سبتمبر", "أكتوبر", "نوفمبر", "ديسمبر",
    "الأحد", "الاثنين", "الثلاثاء", "الأربعاء", "الخميس", "الجمعة", "السبت"
]

In [4]:
st = Stemmer.Stemmer("arabic")


def text_cleaning(text, stemmer=st):
    # remove stop words and punctuation

    text = text.lower()
    text = re.sub(r"\s+", " ", text)
    text = re.sub("(\s\d+)", "", text)
    text = re.sub(r"$\d+\W+|\b\d+\b|\W+\d+$", "", text)
    text = re.sub("\d+", " ", text)
    text = ar.strip_tashkeel(text)
    text = ar.strip_tatweel(text)
    text = text.replace("#", " ")
    text = text.replace("@", " ")
    text = text.replace("_", " ")

    tokenizer = RegexpTokenizer(r"\w+")

    words = tokenizer.tokenize(text)

    stop_words = set(string.punctuation).union(set(arabic_stop_words))

    filtered_list = [word for word in words if word.casefold() not in stop_words]
    
    # word stemming
    stem_words = [stemmer.stemWord(word) for word in filtered_list]

    text = " ".join(map(str, stem_words))
    text = text.replace("آ", "ا")
    text = text.replace("إ", "ا")
    text = text.replace("أ", "ا")
    text = text.replace("ؤ", "و")
    text = text.replace("ئ", "ي")

    return text



df_train["new review_description"] = df_train["review_description"].apply(
    lambda text: text_cleaning(text)
)



df_train.head(20)

,review_description,rating,new review_description
0,شركه زباله و سواقين بتبرشم و مفيش حتي رقم للشك...,-1,شرك زبال سواق تبرشم مفيش حت رقم شكاو سواق يسيب...
1,خدمة الدفع عن طريق الكي نت توقفت عندي اصبح فقط...,1,خدم دفع طريق الك نت توقف عند اصبح فقط دفع نقد
2,تطبيق غبي و جاري حذفه ، عاملين اكواد خصم و لما...,-1,تطبيق غب جار حذف عامل اكواد خصم لما استخدم اكت...
3,فعلا تطبيق ممتاز بس لو فى امكانية يتيح لمستخدم...,1,فعل تطبيق ممتاز بس لو في امكان يتيح لمستخدم تط...
4,سيء جدا ، اسعار رسوم التوصيل لا تمت للواقع ب ص...,-1,سيء جدا اسعار رسوم توصيل تمت واقع ب صله
5,قعد عشرين سنة يدور على سائق بس اما عن توصيل ال...,0,قعد عشرين سنة يدور سايق بس اما توصيل اشياء جيد...
6,احلئ تطبيق,1,احلء تطبيق
7,رائع واو مدهش,1,رايع واو مدهش
8,مکو بس البحرین وعمان وغیرهه بس العراق مکو یعنی...,-1,مکو بس بحری عمان غیر بس عراق مکو یعنی نجم وحد ...
9,تطبيق جميل يستاهل الخمس نجوم👍👍👍,1,تطبيق جميل استاهل خمس نجوم


In [5]:
X = df_train["new review_description"]
y = df_train["rating"].astype(int) + 1

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
total_words = len(tokenizer.word_index) + 1

In [7]:
sequences = tokenizer.texts_to_sequences(X)
max_len = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_len)

In [8]:
X_train, X_val, y_train, y_val = train_test_split(
    padded_sequences, y, test_size=0.2, random_state=42, stratify=y
)

In [9]:
num_classes = 3

In [10]:
model = Sequential()
model.add(
    Embedding(
        input_dim=len(tokenizer.word_index) + 1,
        output_dim=300,
        input_length=X_train.shape[1],
    )
)
model.add(LSTM(50, activation="tanh", dropout=0.5))
model.add(Dense(num_classes, activation="softmax"))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

checkpoint = ModelCheckpoint(
    filepath="./LSTM-emb.hdf5",
    monitor="val_accuracy",
    save_best_only=True,
    save_weights_only=True,
)

In [87]:
model.fit(
    X_train,
    y_train,
    epochs=5,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[checkpoint],
)

Epoch 1/5
801/801 [==============================] - 32s 38ms/step - loss: 0.5301 - accuracy: 0.7985 - val_loss: 0.4729 - val_accuracy: 0.8318
Epoch 2/5
801/801 [==============================] - 29s 36ms/step - loss: 0.3892 - accuracy: 0.8600 - val_loss: 0.4773 - val_accuracy: 0.8258
Epoch 3/5
801/801 [==============================] - 29s 36ms/step - loss: 0.3184 - accuracy: 0.8881 - val_loss: 0.4988 - val_accuracy: 0.8287
Epoch 4/5
801/801 [==============================] - 29s 36ms/step - loss: 0.2673 - accuracy: 0.9058 - val_loss: 0.5389 - val_accuracy: 0.8191
Epoch 5/5
801/801 [==============================] - 29s 36ms/step - loss: 0.2256 - accuracy: 0.9228 - val_loss: 0.5753 - val_accuracy: 0.8166


In [11]:
model.load_weights("./LSTM-emb.hdf5")
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss}, Validation Accuracy: {accuracy}")

201/201 [==============================] - 5s 14ms/step - loss: 0.4516 - accuracy: 0.8429
Validation Loss: 0.4516329765319824, Validation Accuracy: 0.8428527116775513


# TESTING

In [89]:
file_path = "test _no_label.csv"
df_test = pd.read_csv(file_path)
df_test

,ID,review_description
0,1,اهنئكم على خدمه العملاء في المحادثه المباشره م...
1,2,ممتاز جدا ولكن اتمنى ان تكون هناك بعض المسابقا...
2,3,كل محملته يقول تم ايقاف حطيت2 عشان تسوون الخطاء
3,4,شغل طيب
4,5,بعد ماجربت
...,...,...
995,996,يستهل
996,997,خدمة سيئة بكل المعايير
997,998,لؤي٠٣٣٢لؤ٣٤٣س
998,999,تطبيق غير صادق ف خصم الكوبونات


In [90]:
df_test["review_description"] = df_test["review_description"].apply(
    lambda text: text_cleaning(text)
)
X_test_seq = tokenizer.texts_to_sequences(df_test["review_description"])
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

In [91]:
y_pred = model.predict(X_test_pad)

32/32 [==============================] - 1s 13ms/step


In [92]:
prediction = []
for predict in y_pred:
    maxi = np.argmax(predict)
    if maxi == 2:
        prediction.append(1)
    elif maxi == 1:
        prediction.append(0)
    else:
        prediction.append(-1)

results_df = pd.DataFrame({"ID": df_test["ID"], "rating": prediction})
results_df.to_csv("test_results_LSTM_emb.csv", index=False)
results_df

,ID,rating
0,1,1
1,2,1
2,3,-1
3,4,-1
4,5,1
...,...,...
995,996,1
996,997,-1
997,998,1
998,999,-1
